In [1]:
import glob
import os

import cv2
import numpy as np
from sklearn.utils import shuffle

from ContinuousGesturePredictor import define_model


W0718 14:05:09.693006  1844 deprecation_wrapper.py:119] From d:\documents\programming\python\workspace\magic mirror\cnn gesture recognition\venv\lib\site-packages\tflearn\helpers\summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0718 14:05:09.697012  1844 deprecation_wrapper.py:119] From d:\documents\programming\python\workspace\magic mirror\cnn gesture recognition\venv\lib\site-packages\tflearn\helpers\trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0718 14:05:09.702589  1844 deprecation_wrapper.py:119] From d:\documents\programming\python\workspace\magic mirror\cnn gesture recognition\venv\lib\site-packages\tflearn\collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0718 14:05:09.708591  1844 deprecation_wrapper.py:119] From d:\documents\programming\python\workspace\magic mirror\cnn gesture recognition\venv\lib\si

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [14]:
# Load Images
dataset_folder = "dataset/"
num_classes = len(os.listdir(dataset_folder))

loadedImages = []
for filename in glob.iglob(dataset_folder + '/*/train/*', recursive=True):
    if os.path.isfile(filename): # filter dirs
        image = cv2.imread(filename)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        loadedImages.append(np.expand_dims(gray_image, 2))


In [15]:
# Create OutputVector
outputVectors = []
for i in range(len(loadedImages)):
    outputVectors.append([int(k == i // 1000) for k in range(num_classes)])


In [16]:
testImages = []

#Load Images for swing
for filename in glob.iglob(dataset_folder + '/*/test/*', recursive=True):
    if os.path.isfile(filename): # filter dirs
        image = cv2.imread(filename)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        testImages.append(np.expand_dims(gray_image, 2))

testLabels = []

for i in range(len(testImages)):
    testLabels.append([int(k == i // 100) for k in range(num_classes)])

In [19]:
model = define_model()

In [20]:
# Shuffle Training Data
loadedImages, outputVectors = shuffle(loadedImages, outputVectors, random_state=0)

# Train model
model.fit(loadedImages, outputVectors, n_epoch=50,
           validation_set = (testImages, testLabels),
           snapshot_step=100, show_metric=True, run_id='convnet_coursera')

model.save("TrainedModel/GestureRecogModel.tfl")

Training Step: 4699  | total loss: 0.00049 | time: 174.216s
| Adam | epoch: 050 | loss: 0.00049 - acc: 1.0000 -- iter: 5952/6000
Training Step: 4700  | total loss: 0.00045 | time: 180.699s
| Adam | epoch: 050 | loss: 0.00045 - acc: 1.0000 | val_loss: 0.09662 - val_acc: 0.9917 -- iter: 6000/6000
--
